In [ ]:
#Merge sequences 

!qiime feature-table merge-seqs \
--i-data Folder_Apprill_USVIFeb2020/rep_trim_Import_files_Apprill_USVIFeb2020.qza \
--i-data Folder_Apprill_transmissionMucus2019/rep_trim_Import_files_Apprill_2019TransmissionExpMucus.qza \
--i-data folder_Apprill_transmissionTissue2019/rep_trim_Apprill_2019TransmissionExpTissu.qza \
--i-data Folder_Meyer_FRT2017/rep_seqs_Meyer_FRT2017.qza \
--i-data Folder_Muller_EMP/rep_trim_Muller_EMP.qza \
--i-data Folder_SWGmicrobiome2018/rep_trim_demux_Muller_SWGmicro2018.qza \
--i-data Folder_Muller_SWGtransmissionstudies/rep_trim_demux_Muller_SWGtransmissionstudies.qza \
--i-data Folder_SalernoPeters_FDEPSCTLD_2016_2017/rep_seqs_SalernoPeters_FDEPSCTLD_2016_2017.qza \
--i-data Folder_RodriguezLanetty_Bellantuono_SSID_ConchReef_Mar2017/rep_seqs_RodriguezLanetty_Bellantuono_SSID_ConchReef_Mar2017.qza \
--i-data Folder_Schill_LK_2018/rep__Schill_LK_2018.qza \
--i-data Folder_StewartMedina_FL2019/rep_StewartMedina_FL2019.qza \
--i-data Folder_Rosales_AcropraDis/rep_seqs_Rosales_AcropraDis.qza \
--i-data Folder_Voss_ENS_LBTS1/rep_seqs_Voss_ENS_LBTS1.qza \
--o-merged-data rep_merge_SCTLD.qza

In [ ]:
#Merge feature table
!qiime feature-table merge \
--i-tables Folder_Apprill_USVIFeb2020/table_Apprill_USVIFeb2020I.qza \
--i-tables Folder_Apprill_transmissionMucus2019/table_Apprill_2019TransmissionExpMucus.qza \
--i-tables folder_Apprill_transmissionTissue2019/table_Apprill_2019TransmissionExpTissu.qza \
--i-tables Folder_Meyer_FRT2017/table_Meyer_FRT201.qza \
--i-tables Folder_Muller_EMP/table_Muller_EMP_TL.qza \
--i-tables Folder_SWGmicrobiome2018/table_Muller_SWGmicro2018.qza \
--i-tables Folder_Muller_SWGtransmissionstudies/table_Muller_SWGtransmissionstudies.qza \
--i-tables Folder_SalernoPeters_FDEPSCTLD_2016_2017/table_SalernoPeters_FDEPSCTLD_2016_2017.qza \
--i-tables Folder_RodriguezLanetty_Bellantuono_SSID_ConchReef_Mar2017/table_RodriguezLanetty_Bellantuono_SSID_ConchReef_Mar2017.qza \
--i-tables Folder_Schill_LK_2018/table_Schill_LK_2018.qza \
--i-tables Folder_StewartMedina_FL2019/table_StewartMedina_FL2019.qza \
--i-tables Folder_Rosales_AcropraDis/table_Rosales_AcropraDis.qza \
--i-tables Folder_Voss_ENS_LBTS1/table-Voss_ENS_LBTS1.qza \
--o-merged-table table_merge_SCTLD.qza


In [ ]:
# Cluster at 99% identity 
!qiime vsearch cluster-features-de-novo \
--i-sequences rep_merge_SCTLD.qza \
--i-table table_merge_SCTLD.qza --p-perc-identity 0.99 \
--o-clustered-table table-dn-99.qza \
--o-clustered-sequences rep-seqs-dn-99.qza

In [ ]:
# visualize feature table
!qiime feature-table summarize \
  --i-table table-dn-99.qza  \
  --o-visualization table-dn-99.qzv \
  --m-sample-metadata-file metadata/SCTLD_meta_analysis_metadata.txt

In [ ]:
#vsearch to classify sequences with long 16S rRNA seq
!qiime feature-classifier classify-consensus-vsearch \
  --i-reference-taxonomy /space/home/rosales/databases/SILVA_DBs/silva-138-99-tax.qza \
  --i-reference-reads /space/home/rosales/databases/SILVA_DBs/silva-138-99-seqs.qza \
  --i-query rep-seqs-dn-99.qza \
  --p-maxaccepts 3 \
  --p-threads 30 \
  --o-classification taxaVsearch_rep-seqs-dn-99_SCTLD.qza

# visualize
!qiime metadata tabulate \
  --m-input-file taxaVsearch_rep_merge_SCTLD.qza \
  --o-visualization taxaVsearch_rep_merge_SCTLD.qzv

#filter mitochondria and chloroplast
!qiime taxa filter-table \
--i-table table-dn-99.qza \
--i-taxonomy taxaVsearch_rep-seqs-dn-99_SCTLD.qza \
--p-exclude mitochondria,chloroplast \
--p-include d__Bacteria,d__Archaea \
--o-filtered-table tableV_BacArc_99_SCTLD.qza

# visualize
!qiime feature-table summarize \
  --i-table tableV_BacArc_99_SCTLD.qza \
  --o-visualization tableV_BacArc_99_SCTLD.qzv \
  --m-sample-metadata-file metadata/SCTLD_meta_analysis_metadata.txt

#filter mitochondria and chloroplast
!qiime taxa filter-seqs \
--i-sequences rep-seqs-dn-99.qza \
--i-taxonomy  taxaVsearch_rep-seqs-dn-99_SCTLD.qza \
--p-exclude mitochondria,chloroplast \
--p-include d__Bacteria,d__Archaea \
--o-filtered-sequences repV_BacArc_99_SCTLD.qza

In [ ]:
#tree
!qiime fragment-insertion sepp \
  --i-representative-sequences repV_BacArc_99_SCTLD.qza \
  --i-reference-database /space/home/rosales/databases/SILVA_DBs/sepp-refs-silva-128.qza \
  --o-tree insertion-tree99.qza \
  --p-threads 20 \
  --o-placements insertion-placements99.qza

In [ ]:
!qiime fragment-insertion filter-features \
  --i-table tableV_BacArc_99_SCTLD.qza \
  --i-tree insertion-tree99.qza \
  --o-filtered-table tableVfil_BacArc_99_SCTLD.qza \
  --o-removed-table  tableVrem_BacArc_99_SCTLD.qza

In [ ]:
!qiime feature-table summarize \
  --i-table tableVfil_BacArc_99_SCTLD.qza \
  --o-visualization tableVfil_BacArc_99_SCTLD.qzv \
  --m-sample-metadata-file metadata/SCTLD_meta_analysis_metadata.txt


!qiime feature-table summarize \
  --i-table tableVrem_BacArc_99_SCTLD.qza \
  --o-visualization tableVrem_BacArc_99_SCTLD.qzv \
  --m-sample-metadata-file metadata/SCTLD_meta_analysis_metadata.txt

In [ ]:
!qiime tools export \
  --input-path insertion-tree99.qza \
  --output-path exported-tree